# Импорт

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim

# Загрузка данных, преобразование

In [ ]:
data = pd.read_csv('https://drive.google.com/u/0/uc?id=1fbr91UzoCsgAEtCxam_gS8reuULMCpKj&export=download')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
X = data.drop('Outcome', axis=1).values
y = data.Outcome.values

In [ ]:
#нормализация
scal = StandardScaler()
X = scal.fit_transform(X)

In [ ]:
#разделение данный на обучащий и тестовый набор
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=0)

# Создание нейронной сети

In [ ]:
# Определение модели

#для 3 слоев
class NeuralNet3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet3, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = torch.sigmoid(self.fc3(out))

        return out

#для 4 слоев
class NeuralNet4(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet4, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = torch.relu(self.fc1(x))
        out = torch.relu(self.fc2(out))
        out = torch.relu(self.fc3(out))
        out = torch.sigmoid(self.fc4(out))
        return out

# Обучение


### Задание параметров


In [ ]:
learning_rate = 0.001

hiddenlay3 = 200
hiddenlay4 = 400

output_size = 2

### обучаем


In [ ]:
model3 = NeuralNet3(X_train.shape[1], hiddenlay3, output_size)
model4 = NeuralNet4(X_train.shape[1], hiddenlay4, output_size)

In [ ]:
criterion = nn.CrossEntropyLoss()

#Adagrad - Адаптивный оптимизатор, который адаптирует скорость обучения для каждого параметра, учитывая их историю градиентов.
optimizer3 = optim.Adagrad(model3.parameters(), learning_rate)
optimizer4 = optim.Adagrad(model4.parameters(), learning_rate)

In [ ]:
#обучение 3х слойной модели

num_epochs = 30
batch_size = 15

for epoch in range(num_epochs):
  model3.train()
  for i in range(0, len(X_train), batch_size):
    inputs = torch.Tensor(X_train[i:i + batch_size])
    labels = torch.LongTensor(y_train[i:i + batch_size])

    optimizer3.zero_grad() # Обнуление градиентов параметров
    # Прямой проход
    outputs = model3(inputs)
    loss = criterion(outputs, labels)
    # Обратное распространение ошибки и оптимизация
    loss.backward()
    optimizer3.step()

#оценка на тестовом
model3.eval() #режим оценки модели

with torch.no_grad():
  test_inputs = torch.Tensor(X_test)
  predicted_labels3 = torch.argmax(model3(test_inputs), dim=1).numpy()

print('Accuracy = ', accuracy_score(y_test, predicted_labels3))

Accuracy =  0.7987012987012987


In [ ]:
#обучение 4х слойной модели

for epoch in range(num_epochs):
  model4.train()
  for i in range(0, len(X_train), batch_size):
    inputs = torch.Tensor(X_train[i:i + batch_size])
    labels = torch.LongTensor(y_train[i:i + batch_size])

    optimizer4.zero_grad() # Обнуление градиентов параметров
    # Прямой проход
    outputs = model4(inputs)
    loss = criterion(outputs, labels)
    # Обратное распространение ошибки и оптимизация
    loss.backward()
    optimizer4.step()

#оценка на тестовом
model4.eval() #режим оценки модели

with torch.no_grad():
  test_inputs = torch.Tensor(X_test)
  predicted_labels4 = torch.argmax(model4(test_inputs), dim=1).numpy()


print('Accuracy = ', accuracy_score(y_test, predicted_labels4))

Accuracy =  0.7857142857142857
